### Table of Contents

* [Introduction](#Intro)
    * [Problem Statement](#Problem)
* [Dataset Preparation](#DatasetPrep)
    * [Exploratory Data Analysis](#EDA)
* [Customer Categorisation with K-means Clustering](#Clustering)
* [Fine tuning the algorithm](#Tuning)
* [Visualising the results](#DataViz)
* [Interpreting the results](#Results)
* [Conclusions](#Conclusions)

### Introduction

#### Problem Statement
Bussiness case

### Dataset Preparation

#### Dataset

In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go 
from plotly.subplots import make_subplots

from sklearn.cluster import KMeans

%matplotlib inline
%config InlineBackend.figure_format='retina'

pd.options.mode.chained_assignment = None

In [3]:
Order_Records = pd.read_csv("Orders - Analysis Task.csv")

In [4]:
Order_Records.head()

,product_title,product_type,variant_title,variant_sku,variant_id,customer_id,order_id,day,net_quantity,gross_sales,discounts,returns,net_sales,taxes,total_sales,returned_item_quantity,ordered_item_quantity
0,DPR,DPR,100,AD-982-708-895-F-6C894FB,52039657,1312378,83290718932496,04/12/2018,2,200.0,-200.00,0.00,0.0,0.0,0.0,0,2
1,RJF,Product P,28 / A / MTM,83-490-E49-8C8-8-3B100BC,56914686,3715657,36253792848113,01/04/2019,2,190.0,-190.00,0.00,0.0,0.0,0.0,0,2
2,CLH,Product B,32 / B / FtO,68-ECA-BC7-3B2-A-E73DE1B,24064862,9533448,73094559597229,05/11/2018,0,164.8,-156.56,-8.24,0.0,0.0,0.0,-2,2
3,NMA,Product F,40 / B / FtO,6C-1F1-226-1B3-2-3542B41,43823868,4121004,53616575668264,19/02/2019,1,119.0,-119.00,0.00,0.0,0.0,0.0,0,1
4,NMA,Product F,40 / B / FtO,6C-1F1-226-1B3-2-3542B41,43823868,4121004,29263220319421,19/02/2019,1,119.0,-119.00,0.00,0.0,0.0,0.0,0,1


In [5]:
Order_Records.shape
print(f"The raw data has {Order_Records.shape[0]} order records with {Order_Records.shape[1]} variables describing each order record")

The raw data has 70052 order records with 17 variables describing each order record


In [6]:
Order_Records.info() #Overview of the customer order records

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70052 entries, 0 to 70051
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   product_title           70052 non-null  object 
 1   product_type            70052 non-null  object 
 2   variant_title           70052 non-null  object 
 3   variant_sku             70052 non-null  object 
 4   variant_id              70052 non-null  int64  
 5   customer_id             70052 non-null  int64  
 6   order_id                70052 non-null  int64  
 7   day                     70052 non-null  object 
 8   net_quantity            70052 non-null  int64  
 9   gross_sales             70052 non-null  float64
 10  discounts               70052 non-null  float64
 11  returns                 70052 non-null  float64
 12  net_sales               70052 non-null  float64
 13  taxes                   70052 non-null  float64
 14  total_sales             70052 non-null

#### Exploratory Data Analysis

In [7]:
#Check for missing NA values

In [8]:
Order_Records.describe() #Summary of each independent variable

,variant_id,customer_id,order_id,net_quantity,gross_sales,discounts,returns,net_sales,taxes,total_sales,returned_item_quantity,ordered_item_quantity
count,7.005200e+04,7.005200e+04,7.005200e+04,70052.000000,70052.000000,70052.000000,70052.000000,70052.000000,70052.000000,70052.000000,70052.000000,70052.000000
mean,2.442320e+11,6.013091e+11,5.506075e+13,0.701179,61.776302,-4.949904,-10.246051,46.580348,9.123636,55.703982,-0.156098,0.857277
std,4.255079e+12,6.223201e+12,2.587640e+13,0.739497,31.800689,7.769972,25.154677,51.802690,10.305236,61.920557,0.369190,0.380820
min,1.001447e+07,1.000661e+06,1.000657e+13,-3.000000,0.000000,-200.000000,-237.500000,-237.500000,-47.500000,-285.000000,-3.000000,0.000000
25%,2.692223e+07,3.295695e+06,3.270317e+13,1.000000,51.670000,-8.340000,0.000000,47.080000,8.375000,56.227500,0.000000,1.000000
50%,4.494514e+07,5.566107e+06,5.522207e+13,1.000000,74.170000,0.000000,0.000000,63.330000,12.660000,76.000000,0.000000,1.000000
75%,7.743106e+07,7.815352e+06,7.736876e+13,1.000000,79.170000,0.000000,0.000000,74.170000,14.840000,89.000000,0.000000,1.000000
max,8.422212e+13,9.977409e+13,9.999554e+13,6.000000,445.000000,0.000000,0.000000,445.000000,63.340000,445.000000,0.000000,6.000000


From the summary of all independent variables, few interesting observations
1. returned item quantity is zero or negative for all the records
2. net quantity is zero or negative for significant number of records
3. ordered_item_quantity is zero for some records

table the data, draw some graphs and do shit here

In [9]:
Cust_NQZ = Order_Records.loc[Order_Records['net_quantity'] == 0] #order records with net quantity zero
Cust_NQLZ = Order_Records.loc[Order_Records['net_quantity'] < 0]

print(f"There are {Cust_NQZ.shape[0]} order records with net_quantity as zero")
print(f"There are {Cust_NQLZ.shape[0]} order records with net_quantity less than zero")

There are 68 order records with net_quantity as zero
There are 10715 order records with net_quantity less than zero


In [10]:
Cust_RIQZ = Order_Records.loc[Order_Records['returned_item_quantity'] == 0] #order records with returned_item_quantity zero
Cust_RIQLZ = Order_Records.loc[Order_Records['returned_item_quantity'] < 0]

print(f"There are {Cust_RIQZ.shape[0]} order records with returned_item_quantity as zero")
print(f"There are {Cust_RIQLZ.shape[0]} order records with returned_item_quantity less than zero")

There are 59269 order records with returned_item_quantity as zero
There are 10783 order records with returned_item_quantity less than zero


In [11]:
Cust_OIQZ = Order_Records.loc[Order_Records['ordered_item_quantity'] == 0] #order records with ordered_item_quantity zero
Cust_OIQLZ = Order_Records.loc[Order_Records['ordered_item_quantity'] < 0]

print(f"There are {Cust_OIQZ.shape[0]} order records with ordered_item_quantity as zero")
print(f"There are {Cust_OIQLZ.shape[0]} order records with ordered_item_quantity less than zero")

There are 10715 order records with ordered_item_quantity as zero
There are 0 order records with ordered_item_quantity less than zero


for significant number of rows, ordered quantity is zero, we need to further investigate as to why the orders have ordered quantity as zero
net quantity is less tan zero. it may indicate that these are erroneous data, need to investigate further to check

In [12]:
Cust_NQLZ.head()

,product_title,product_type,variant_title,variant_sku,variant_id,customer_id,order_id,day,net_quantity,gross_sales,discounts,returns,net_sales,taxes,total_sales,returned_item_quantity,ordered_item_quantity
59295,NMA,Product F,40 / B / FtO,6C-1F1-226-1B3-2-3542B41,43823868,4121004,13666410519728,01/03/2019,-1,0.0,0.0,0.0,0.0,0.0,0.0,-1,0
59300,NMA,Product F,40 / B / FtO,6C-1F1-226-1B3-2-3542B41,43823868,4121004,53616575668264,23/02/2019,-1,0.0,0.0,0.0,0.0,0.0,0.0,-1,0
59305,RJF,Product T,28 / A / 9,4D-D1F-A14-8D9-0-FD0E84A,31355561,3715657,93146430228825,04/12/2018,-1,0.0,0.0,0.0,0.0,0.0,0.0,-1,0
59314,OTH,Product F,40 / B / FtO,53-5CA-7CF-8F5-9-28CB78B,43823868,4121004,53616575668264,23/02/2019,-1,0.0,0.0,0.0,0.0,0.0,0.0,-1,0
59328,YQX,Product H,40 / B / FtO,F2-055-4C3-8C3-0-7070F1D,25826279,4121004,13666410519728,01/03/2019,-1,0.0,0.0,0.0,0.0,0.0,0.0,-1,0


In [13]:
Cust_OIQZ.head()

,product_title,product_type,variant_title,variant_sku,variant_id,customer_id,order_id,day,net_quantity,gross_sales,discounts,returns,net_sales,taxes,total_sales,returned_item_quantity,ordered_item_quantity
59295,NMA,Product F,40 / B / FtO,6C-1F1-226-1B3-2-3542B41,43823868,4121004,13666410519728,01/03/2019,-1,0.0,0.0,0.0,0.0,0.0,0.0,-1,0
59300,NMA,Product F,40 / B / FtO,6C-1F1-226-1B3-2-3542B41,43823868,4121004,53616575668264,23/02/2019,-1,0.0,0.0,0.0,0.0,0.0,0.0,-1,0
59305,RJF,Product T,28 / A / 9,4D-D1F-A14-8D9-0-FD0E84A,31355561,3715657,93146430228825,04/12/2018,-1,0.0,0.0,0.0,0.0,0.0,0.0,-1,0
59314,OTH,Product F,40 / B / FtO,53-5CA-7CF-8F5-9-28CB78B,43823868,4121004,53616575668264,23/02/2019,-1,0.0,0.0,0.0,0.0,0.0,0.0,-1,0
59328,YQX,Product H,40 / B / FtO,F2-055-4C3-8C3-0-7070F1D,25826279,4121004,13666410519728,01/03/2019,-1,0.0,0.0,0.0,0.0,0.0,0.0,-1,0


It looks like, the net quantity is less than zero when the returned item quantity is negative. To understand if the negative returned item quantity being negative is erroneous or not, we need to further investigate into the data

In [14]:
#Lets take record 59300 and check 
customerx = Order_Records.loc[Order_Records['customer_id']==4121004]
customerx

,product_title,product_type,variant_title,variant_sku,variant_id,customer_id,order_id,day,net_quantity,gross_sales,discounts,returns,net_sales,taxes,total_sales,returned_item_quantity,ordered_item_quantity
3,NMA,Product F,40 / B / FtO,6C-1F1-226-1B3-2-3542B41,43823868,4121004,53616575668264,19/02/2019,1,119.0,-119.0,0.0,0.0,0.0,0.0,0,1
4,NMA,Product F,40 / B / FtO,6C-1F1-226-1B3-2-3542B41,43823868,4121004,29263220319421,19/02/2019,1,119.0,-119.0,0.0,0.0,0.0,0.0,0,1
5,OTH,Product F,40 / B / FtO,53-5CA-7CF-8F5-9-28CB78B,43823868,4121004,53616575668264,19/02/2019,1,119.0,-119.0,0.0,0.0,0.0,0.0,0,1
6,OTH,Product F,40 / B / FtO,53-5CA-7CF-8F5-9-28CB78B,43823868,4121004,29263220319421,19/02/2019,1,119.0,-119.0,0.0,0.0,0.0,0.0,0,1
7,NMA,Product F,40 / B / FtO,6C-1F1-226-1B3-2-3542B41,43823868,4121004,13666410519728,20/02/2019,1,119.0,-119.0,0.0,0.0,0.0,0.0,0,1
8,OTH,Product F,40 / C / FtO,8B-2C5-548-6C6-E-B5EECBC,43823868,4121004,80657249973427,22/02/2019,1,119.0,-119.0,0.0,0.0,0.0,0.0,0,1
16,WHX,Product P,40 / C / FtO,44-893-E04-6EF-F-E418295,14526828,4121004,80657249973427,22/02/2019,1,95.0,-95.0,0.0,0.0,0.0,0.0,0,1
19,WHX,Product P,40 / B / FtO,AC-93B-065-BD2-A-5D62CD8,14526828,4121004,29263220319421,19/02/2019,1,95.0,-95.0,0.0,0.0,0.0,0.0,0,1
24,YQX,Product H,40 / B / FtO,F2-055-4C3-8C3-0-7070F1D,25826279,4121004,13666410519728,20/02/2019,1,89.0,-89.0,0.0,0.0,0.0,0.0,0,1
27,YQX,Product H,40 / B / FtO,F2-055-4C3-8C3-0-7070F1D,25826279,4121004,29263220319421,19/02/2019,1,89.0,-89.0,0.0,0.0,0.0,0.0,0,1


take record 59300 and record 4, record 4 predates record 59300. This implies that, the returned items are marked with negative sign and are not erroneous data records. For these records, net quantity is less than zero as the customer has not ordered anything but returned something.

In [15]:
Cust_NQZ.head() #orders where net quantity is zero

,product_title,product_type,variant_title,variant_sku,variant_id,customer_id,order_id,day,net_quantity,gross_sales,discounts,returns,net_sales,taxes,total_sales,returned_item_quantity,ordered_item_quantity
2,CLH,Product B,32 / B / FtO,68-ECA-BC7-3B2-A-E73DE1B,24064862,9533448,73094559597229,05/11/2018,0,164.8,-156.56,-8.24,0.0,0.0,0.0,-2,2
22,KNB,Product H,28 / B / 29,BA-184-06C-4E3-6-1DC738F,10434338,1481447,82857371444896,09/11/2018,0,89.0,-89.00,0.00,0.0,0.0,0.0,-1,1
23,EYV,Product H,31 / A / FtO,E5-666-054-F18-A-90F6B20,22559066,3619130,88025805105285,13/11/2018,0,89.0,-89.00,0.00,0.0,0.0,0.0,-1,1
30,WHX,Product P,32 / B / FtO,85-2EB-163-D62-5-FC50316,26246865,9533448,73094559597229,05/11/2018,0,74.2,-70.49,-3.71,0.0,0.0,0.0,-1,1
31,KNB,Product P,32 / B / FtO,C5-B40-3CE-CB1-9-672218E,30277881,9533448,12837914491890,05/11/2018,0,74.2,-70.49,-3.71,0.0,0.0,0.0,-1,1


these are the records where the ordered quanity and returned quanity is same. i.e, the order has been cancelled even before it has been placed and checked out. These records can be removed as they are not usefull for us

In [16]:

print(f"There are {Cust_NQZ.shape[0]} order records with net_quantity as zero")

There are 68 order records with net_quantity as zero


In [17]:
Order_Records = Order_Records.loc[Order_Records['net_quantity']!=0] #Removing records with net_quanity = 0

In [18]:
Order_Records.shape 

(69984, 17)

#### Feature Engineering
Features to create - number of products ordered, average return rate and total spending

#### Feature 1 : Items Ordered
total number of items ordered by the customer

In [19]:
agg_records_1 = (Order_Records.groupby('customer_id').ordered_item_quantity.sum().reset_index())

In [20]:
agg_records_1.head()

,customer_id,ordered_item_quantity
0,1000661,3
1,1001914,1
2,1002167,3
3,1002387,1
4,1002419,2


#### Feature 2 : Product types Ordered
total number of different product types ordered by the customer

In [21]:
agg_records_2 = (Order_Records.groupby(['customer_id','product_type']).ordered_item_quantity.count().reset_index())

In [22]:
agg_records_2.head()

,customer_id,product_type,ordered_item_quantity
0,1000661,Product P,5
1,1001914,Product B,1
2,1002167,Product H,1
3,1002167,Product P,2
4,1002167,Product W,1


In [23]:
agg_records_2 = agg_records_2.groupby("customer_id").product_type.count().reset_index()

In [24]:
agg_records_2.head()

,customer_id,product_type
0,1000661,1
1,1001914,1
2,1002167,3
3,1002387,1
4,1002419,2


In [25]:
agg_records_1.shape #unique customers

(25530, 2)

In [26]:
agg_records_2.shape #unique customers

(25530, 2)

#### Feature 3 : Average return rate
ratio of total number of orders to the total number of returns by the customer

In [174]:
agg_records_3 = (Order_Records.groupby('customer_id').returned_item_quantity.sum().reset_index())

In [175]:
agg_records_3.head()

,customer_id,returned_item_quantity
0,1000661,-2
1,1001914,0
2,1002167,-1
3,1002387,0
4,1002419,-1


In [176]:
agg_records_4 = pd.merge(agg_records_1,agg_records_3)

In [177]:
agg_records_4.head()

,customer_id,ordered_item_quantity,returned_item_quantity
0,1000661,3,-2
1,1001914,1,0
2,1002167,3,-1
3,1002387,1,0
4,1002419,2,-1


In [178]:
agg_records_4["avg_return_rate"] = (-1*agg_records_4["returned_item_quantity"])/agg_records_4["ordered_item_quantity"]

In [179]:
agg_records_4.head()

,customer_id,ordered_item_quantity,returned_item_quantity,avg_return_rate
0,1000661,3,-2,0.666667
1,1001914,1,0,0.000000
2,1002167,3,-1,0.333333
3,1002387,1,0,0.000000
4,1002419,2,-1,0.500000


In [180]:
agg_records_4.shape

(25530, 4)

#### Feature 4 : Total Spending by the customer
sum of total spending (sales after taxes and returns) of each customer

In [34]:
agg_records_5 = Order_Records.groupby("customer_id").total_sales.sum().reset_index()

In [35]:
agg_records_5.head()

,customer_id,total_sales
0,1000661,86.67
1,1001914,79.20
2,1002167,167.45
3,1002387,89.00
4,1002419,53.10


#### Feature 5 : Average Spending per order by the customer
ratio of total spending to number of total orders (ordered quantity - returned quantity)

In [36]:
agg_records_6 = pd.merge(agg_records_4,agg_records_5)

In [37]:
agg_records_6.head()

,customer_id,ordered_item_quantity,returned_item_quantity,avg_return_rate,total_sales
0,1000661,3,-2,0.666667,86.67
1,1001914,1,0,0.000000,79.20
2,1002167,3,-1,0.333333,167.45
3,1002387,1,0,0.000000,89.00
4,1002419,2,-1,0.500000,53.10


In [38]:
agg_records_6["actual_orders"] = agg_records_6['ordered_item_quantity']+agg_records_6['returned_item_quantity']

In [39]:
agg_records_6.head()

,customer_id,ordered_item_quantity,returned_item_quantity,avg_return_rate,total_sales,actual_orders
0,1000661,3,-2,0.666667,86.67,1
1,1001914,1,0,0.000000,79.20,1
2,1002167,3,-1,0.333333,167.45,2
3,1002387,1,0,0.000000,89.00,1
4,1002419,2,-1,0.500000,53.10,1


In [40]:
agg_records_6["average_spending_per_order"] = agg_records_6['total_sales']/agg_records_6['actual_orders']

In [108]:
agg_records_6.head()

,customer_id,ordered_item_quantity,returned_item_quantity,avg_return_rate,total_sales,actual_orders,average_spending_per_order
0,1000661,3,-2,0.666667,86.67,1,86.670
1,1001914,1,0,0.000000,79.20,1,79.200
2,1002167,3,-1,0.333333,167.45,2,83.725
3,1002387,1,0,0.000000,89.00,1,89.000
4,1002419,2,-1,0.500000,53.10,1,53.100


In [182]:
agg_records_7 = pd.merge(agg_records_2,agg_records_6)

In [183]:
agg_records_7.head()

,customer_id,product_type,ordered_item_quantity,returned_item_quantity,avg_return_rate,total_sales,actual_orders,average_spending_per_order
0,1000661,1,3,-2,0.666667,86.67,1,86.670
1,1001914,1,1,0,0.000000,79.20,1,79.200
2,1002167,3,3,-1,0.333333,167.45,2,83.725
3,1002387,1,1,0,0.000000,89.00,1,89.000
4,1002419,2,2,-1,0.500000,53.10,1,53.100


In [184]:
agg_records_8 = agg_records_7.loc[agg_records_7['ordered_item_quantity']>0]

In [185]:
agg_records_8.rename(columns={"product_type":"types_of_products_ordered"},inplace=True)
agg_records_8.rename(columns={"ordered_item_quantity":"total_items_ordered"},inplace=True)
agg_records_8.rename(columns={"total_sales":"total_spending"},inplace=True)
agg_records_8.rename(columns={"average_spending_per_order":"average_spending"},inplace=True)


In [186]:
agg_records_8.head()

,customer_id,types_of_products_ordered,total_items_ordered,returned_item_quantity,avg_return_rate,total_spending,actual_orders,average_spending
0,1000661,1,3,-2,0.666667,86.67,1,86.670
1,1001914,1,1,0,0.000000,79.20,1,79.200
2,1002167,3,3,-1,0.333333,167.45,2,83.725
3,1002387,1,1,0,0.000000,89.00,1,89.000
4,1002419,2,2,-1,0.500000,53.10,1,53.100


In [188]:
agg_records_8.describe()

,customer_id,types_of_products_ordered,total_items_ordered,returned_item_quantity,avg_return_rate,total_spending,actual_orders,average_spending
count,2.486100e+04,24861.000000,24861.000000,24861.000000,24861.000000,24861.000000,24861.000000,23201.0
mean,6.569737e+11,1.751297,2.412775,-0.399139,0.143447,160.016373,2.013636,NaN
std,6.562048e+12,1.093116,2.165090,0.989452,0.314892,148.504786,1.869392,NaN
min,1.000661e+06,1.000000,1.000000,-26.000000,0.000000,-657.300000,-7.000000,-inf
25%,3.295365e+06,1.000000,1.000000,0.000000,0.000000,75.000000,1.000000,70.0
50%,5.578765e+06,1.000000,2.000000,0.000000,0.000000,108.000000,1.000000,80.1
75%,7.845941e+06,2.000000,3.000000,0.000000,0.000000,197.000000,2.000000,92.0
max,9.954992e+13,13.000000,39.000000,0.000000,8.000000,2304.300000,28.000000,inf


In [214]:
agg_records_9 = agg_records_8.loc[agg_records_8['actual_orders']>0]


In [216]:
agg_records_10 = agg_records_9.loc[agg_records_8['total_spending']>0]

In [217]:
agg_records_10.describe()

,customer_id,types_of_products_ordered,total_items_ordered,returned_item_quantity,avg_return_rate,total_spending,actual_orders,average_spending
count,2.298300e+04,22983.000000,22983.000000,22983.000000,22983.000000,22983.000000,22983.000000,22983.000000
mean,6.430829e+11,1.783840,2.482052,-0.299265,0.070184,173.531391,2.182787,80.458827
std,6.497248e+12,1.114382,2.214307,0.894915,0.170588,146.023162,1.838998,16.637222
min,1.000661e+06,1.000000,1.000000,-26.000000,0.000000,4.450000,1.000000,4.450000
25%,3.290167e+06,1.000000,1.000000,0.000000,0.000000,85.500000,1.000000,70.000000
50%,5.571368e+06,1.000000,2.000000,0.000000,0.000000,128.200000,2.000000,80.100000
75%,7.848048e+06,2.000000,3.000000,0.000000,0.000000,213.000000,3.000000,92.000000
max,9.954992e+13,13.000000,39.000000,0.000000,0.928571,2304.300000,28.000000,178.000000


In [218]:
agg_records_10.drop(["returned_item_quantity","actual_orders"],axis=1,inplace=True)

In [219]:
Customer_Records = agg_records_10.copy(deep=True)

In [220]:
Customer_Records.head()

,customer_id,types_of_products_ordered,total_items_ordered,avg_return_rate,total_spending,average_spending
0,1000661,1,3,0.666667,86.67,86.670
1,1001914,1,1,0.000000,79.20,79.200
2,1002167,3,3,0.333333,167.45,83.725
3,1002387,1,1,0.000000,89.00,89.000
4,1002419,2,2,0.500000,53.10,53.100


In [221]:
Customer_Records.describe()

,customer_id,types_of_products_ordered,total_items_ordered,avg_return_rate,total_spending,average_spending
count,2.298300e+04,22983.000000,22983.000000,22983.000000,22983.000000,22983.000000
mean,6.430829e+11,1.783840,2.482052,0.070184,173.531391,80.458827
std,6.497248e+12,1.114382,2.214307,0.170588,146.023162,16.637222
min,1.000661e+06,1.000000,1.000000,0.000000,4.450000,4.450000
25%,3.290167e+06,1.000000,1.000000,0.000000,85.500000,70.000000
50%,5.571368e+06,1.000000,2.000000,0.000000,128.200000,80.100000
75%,7.848048e+06,2.000000,3.000000,0.000000,213.000000,92.000000
max,9.954992e+13,13.000000,39.000000,0.928571,2304.300000,178.000000
